<a href="https://colab.research.google.com/github/christie87/offsyo/blob/main/Simple_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
#!pip install pyngrok
#!pip install dash

In [99]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc,html 
from dash.dependencies import Input, Output, State
from dash import no_update

In [100]:
from pyngrok import ngrok
ngrok.kill()
NGROK_AUTH_TOKEN = "..."      #enter personal ngrok auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
dash_connect=ngrok.connect(8050)   #access dashboard via generated link

In [101]:
df=pd.read_csv('https://raw.githubusercontent.com/christie87/offsyo/main/US_zomato_data.csv')
df.head()

,Unnamed: 0,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Price range,Aggregate rating,Rating text,Votes
0,0,17284404,Austin's BBQ and Oyster Bar,216,Albany,"2820 Meredyth Dr, Albany, GA 31707",Albany,-84.221535,31.610387,"BBQ, Burger, Seafood",25,Dollar($),2,3.3,Average,35
1,1,17284203,BJ's Country Buffet,216,Albany,"2401 Dawson Rd, Albany, GA 31707",Albany,-84.207095,31.608743,"American, BBQ",10,Dollar($),1,3.3,Average,25
2,2,17284397,Elements Coffee Co - Northwest,216,Albany,"2726 Ledo Rd Ste 10, Albany, GA 31707",Albany,-84.206944,31.622412,"Coffee and Tea, Sandwich",10,Dollar($),1,3.4,Average,26
3,3,17284094,Chick-fil-A,216,Albany,"2703 Dawson Rd, Albany, GA 31707",Albany,-84.219300,31.616000,Fast Food,10,Dollar($),1,3.5,Good,67
4,4,17284409,Guang Zhou Chinese Restaurant,216,Albany,"1214 N Westover Blvd, Albany, GA 31707",Albany,-84.209146,31.615519,"Asian, Chinese, Vegetarian",10,Dollar($),1,3.9,Good,141


In [102]:
df.drop(columns='Unnamed: 0',inplace=True)
df.columns = list(map(str, df.columns))
df.duplicated().value_counts()

False    415
dtype: int64

In [103]:
df.shape

(415, 15)

In [104]:
df["Cuisines"] = df["Cuisines"].str.split(", ")

# Explode the entire data frame
df = df.apply( pd.Series.explode)
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Price range,Aggregate rating,Rating text,Votes
0,17284404,Austin's BBQ and Oyster Bar,216,Albany,"2820 Meredyth Dr, Albany, GA 31707",Albany,-84.221535,31.610387,BBQ,25,Dollar($),2,3.3,Average,35
0,17284404,Austin's BBQ and Oyster Bar,216,Albany,"2820 Meredyth Dr, Albany, GA 31707",Albany,-84.221535,31.610387,Burger,25,Dollar($),2,3.3,Average,35
0,17284404,Austin's BBQ and Oyster Bar,216,Albany,"2820 Meredyth Dr, Albany, GA 31707",Albany,-84.221535,31.610387,Seafood,25,Dollar($),2,3.3,Average,35
1,17284203,BJ's Country Buffet,216,Albany,"2401 Dawson Rd, Albany, GA 31707",Albany,-84.207095,31.608743,American,10,Dollar($),1,3.3,Average,25
1,17284203,BJ's Country Buffet,216,Albany,"2401 Dawson Rd, Albany, GA 31707",Albany,-84.207095,31.608743,BBQ,10,Dollar($),1,3.3,Average,25


In [ ]:
cuisine_list=df['Cuisines'].unique().tolist()
cuisine_list

In [106]:
df.loc[df['Restaurant Name']=='Shorts Burger and Shine']


,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Price range,Aggregate rating,Rating text,Votes
86,17316744,Shorts Burger and Shine,216,Cedar Rapids/Iowa City,"18 S Clinton St, Iowa City, IA 52240",Iowa City,-91.534424,41.660982,Burger,10,Dollar($),1,4.9,Excellent,820


In [107]:
dash_connect #generated dashboard link

<NgrokTunnel: "http://2adf-34-91-104-215.ngrok.io" -> "http://localhost:8050">

In [108]:
# Create a dash application
app = dash.Dash(__name__,suppress_callback_exceptions = True)


# Build dash app layout
app.layout = html.Div(children=[ html.H1("Zomato Restaurants by Cuisine", 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 35}),
                                html.Div(["Select Cuisine: ", dcc.Dropdown(id='cuisine-type',
                         
                         options=[{'label': i, 'value': i} for i in cuisine_list],
                         placeholder="Cuisine Choice",
                         style={'width': '80%', 'padding': '3px', 'font-size': '25px', 'textAlign': 'center'}),], 
                                style={'font-size': 20}),
                                html.Br(),
                                html.Br(), 
                               
                                html.Div([
                                        html.Div([ ], id='sunfig'),
                                        html.Div([ ], id='bubfig')
                                ], style={'display': 'flex'}),
                                
                                html.Div([],id='barfig'),
                                html.Br(),
                                html.Br(),

                                html.Div(html.P('datasource::Kaggle.com::Zomato Restaurants Data'),
                                         style={'textAlign': 'right', 'color': '#503D36',
                                         'font-size': 14})

])
# Callback decorator
@app.callback( [
               Output(component_id='sunfig', component_property='children'),
               Output(component_id='bubfig', component_property='children'),
               Output(component_id='barfig', component_property='children')
               ],
               Input(component_id='cuisine-type', component_property='value'),
               prevent_initial_call=True)
# Computation to callback function and return graph
def display_selected_cuisine(value):
    us = df[df['Cuisines']==value].groupby(['City','Rating text','Restaurant Name'],as_index=False).mean()  
    
    sunfig = px.sunburst(us, path=['Rating text','City'], values='Aggregate rating',
                         color='Rating text',title='Rated Restaurants in the USA', 
                         color_discrete_sequence=px.colors.qualitative.Bold)
    bubfig = px.scatter(us, x="Aggregate rating", y="Rating text", size="Votes", color='Restaurant Name',
                       size_max=60,title='Rated Restaurants by Votes',
                       color_discrete_sequence=px.colors.qualitative.Alphabet)
    barfig=px.bar(us[us['Rating text']=='Excellent'],x='City',y='Aggregate rating',
                  color='Restaurant Name',title='Restaurants Rated \'Excellent\'',
                  barmode='group',opacity=0.7,
                  color_discrete_sequence=px.colors.qualitative.Bold)
    barfig.update_traces(width=0.5)

    
    return [dcc.Graph(figure=sunfig),
            dcc.Graph(figure=bubfig),
            dcc.Graph(figure=barfig) ]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8050
INFO:werkzeug:Press CTRL+C to quit
